# Marlan McInnes-Taylor, mm05f

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class tispSelector:
    def __init__(self, xTrain, yTrain, xValid, yValid, lambdas, maxItr=100):
        self.xTrain = xTrain
        self.yTrain = yTrain
        self.xValid = xValid
        self.yValid = yValid
        self.preProcess()
        self.w = np.zeros(self.xTrain.shape[1])
        self.lam = 0.0
        self.lambdas = lambdas
        self.eta = 1./self.xTrain.shape[0]
        self.maxItr = maxItr

    def preProcess(self):
        xStd = np.std(self.xTrain, axis=0)
        mask = (xStd != 0.)
        self.xTrain = self.xTrain[:, mask]
        meanX = np.mean(self.xTrain, axis=0)
        stdX = np.std(self.xTrain, axis=0)
        self.xTrain = (self.xTrain - meanX)/stdX
        self.xValid = self.xValid[:, mask]
        self.xValid = (self.xValid - meanX)/stdX

        self.xTrain = np.insert(self.xTrain, 0, 1., axis=1)
        self.xValid = np.insert(self.xValid, 0, 1., axis=1)

        self.yTrain[self.yTrain == 0.] =- 1.
        self.yValid[self.yValid == 0.] =- 1.

    def gradient(self):
        wx = np.sum(self.xTrain * self.w, axis=1)
        temp = self.yTrain/(1. + np.exp(self.yTrain * wx))
        grad = np.sum(temp * (self.xTrain).T, axis=1)
        return grad

    def update(self):
        grad = self.gradient()
        self.w += grad * self.eta
        self.w[np.absolute(self.w)<=self.lam] = 0.

    def train(self):        
        testErrors = np.zeros(len(self.lambdas))
        trainErrors = np.zeros(len(self.lambdas))
        features = np.zeros(len(self.lambdas))
        
        for j in range(len(self.lambdas)):
            self.lam = self.lambdas[j]

            for i in range(self.maxItr):
                self.update()
                # print(i, 'Number of features: ', np.sum(self.w!=0.))

            wx = np.sum(self.xTrain * self.w, axis=1)
            pred=np.ones(self.yTrain.shape[0])
            pred[wx<0.] =- 1.
            trainErrors[j] = 1. - np.mean(pred==self.yTrain)
            features[j] = np.sum(self.w!=0.)

            wx = np.sum(self.xValid * self.w, axis=1)
            pred = np.ones(self.yValid.shape[0])
            pred[wx<0.] =- 1.
            testErrors[j]= 1.- np.mean(pred==self.yValid)
            '''print('Lambda:', self.lam, '| Features:', features[j], '| Training Error:',
                   trainErrors[j], '| Test Error:', testErrors[j])'''
            self.w = np.zeros_like(self.w)
            
        return trainErrors, testErrors, features 

In [ ]:
def graphResults(trainErrors, testErrors, numFeatures):
    trainErrors = trainErrors * 100
    testErrors = testErrors * 100

    plt.plot(numFeatures, trainErrors, linestyle='-', marker='o', color='r', label='Train')
    plt.plot(numFeatures, testErrors, linestyle='-', marker='o', color='b', label='Test')
    plt.grid(True)
    plt.xticks(numFeatures)
    plt.title('Misclassification Error vs Number of Features')
    plt.xlabel('Number of Features')
    plt.ylabel('Misclassification Error (%)')
    plt.legend()
    plt.show()
    
def genTable(trainErrors, testErrors, features, lambdas):
    lambdas = np.array(lambdas)
    trainErrors = trainErrors * 100
    testErrors = testErrors * 100
    
    errorTable = pd.DataFrame({"Features":[0, 0, 0, 0, 0], "Training Error (%)":[0, 0, 0, 0, 0], "Test Error (%)":[0, 0, 0, 0, 0]}, index=lambdas)
    errorTable.index.name = "Lambda"

    for (lam, train, test, numFeatures) in zip(lambdas, trainErrors, testErrors, features):
        errorTable.loc[lam, 'Features'] = numFeatures
        errorTable.loc[lam, 'Training Error (%)'] = round(train, 2)
        errorTable.loc[lam, 'Test Error (%)'] = round(test, 2)
    
    return errorTable

# Part A

In [ ]:
xTrain = np.loadtxt('../data/Gisette/gisette_train.data')
yTrain = np.loadtxt('../data/Gisette/gisette_train.labels')
xValid = np.loadtxt('../data/Gisette/gisette_valid.data')
yValid = np.loadtxt('../data/Gisette/gisette_valid.labels')

In [ ]:
lambdas = [0.188, 0.135, 0.0876, 0.053, 0.03855]

model = tispSelector(xTrain, yTrain, xValid, yValid, lambdas)
trainErrors, testErrors, features = model.train()

In [ ]:
errorGis = genTable(trainErrors, testErrors, features, lambdas)
graphResults(trainErrors, testErrors, features)

# Part B

In [ ]:
xTrain = np.loadtxt('../data/dexter/dexter_train.csv', delimiter=',')
yTrain = np.loadtxt('../data/dexter/dexter_train.labels')
xValid = np.loadtxt('../data/dexter/dexter_valid.csv', delimiter=',')
yValid = np.loadtxt('../data/dexter/dexter_valid.labels')

In [ ]:
lambdas = [0.142, 0.099, 0.0712, 0.0523, 0.0466]

model = tispSelector(xTrain, yTrain, xValid, yValid, lambdas)
trainErrors, testErrors, features = model.train()

In [ ]:
errorDex = genTable(trainErrors, testErrors, features, lambdas)
graphResults(trainErrors, testErrors, features)

# Part C

In [ ]:
xTrain = np.loadtxt('../data/MADELON/madelon_train.data')
yTrain = np.loadtxt('../data/MADELON/madelon_train.labels')
xValid = np.loadtxt('../data/MADELON/madelon_valid.data')
yValid = np.loadtxt('../data/MADELON/madelon_valid.labels')

In [ ]:
lambdas = [0.0297959738, 0.0246, 0.01775, 0.0075, 0.0001]

model = tispSelector(xTrain, yTrain, xValid, yValid, lambdas)
trainErrors, testErrors, features = model.train()

In [ ]:
errorMad = genTable(trainErrors, testErrors, features, lambdas)
graphResults(trainErrors, testErrors, features)

In [ ]:
errorTableFull = pd.concat([errorGis, errorDex, errorMad])
errorTableFull